In [1]:
import matplotlib.pyplot as plt
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer, LabelBinarizer
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf
import pickle
import numpy as np
import pandas as pd
from random import choice

from annsa.template_sampling import *

In [2]:
import tensorflow.contrib.eager as tfe

In [3]:
tf.enable_eager_execution()

#### Import model, training function 

In [20]:
from annsa.model_classes import (cnn1d_model_features,
                                 CNN1D,
                                 generate_random_cnn1d_architecture,
                                 save_model,
                                 train_earlystop)

## Dataset Construction

#### Load dataset

In [5]:
background_dataset = pd.read_csv('../../source-interdiction/training_testing_data/background_template_dataset.csv')
source_dataset = pd.read_csv('../../source-interdiction/training_testing_data/shielded_templates_200kev_dataset.csv')

#### Dataset details

In [6]:
print('sourcedist: ' + str(sorted(set(source_dataset['sourcedist']))))
print('sourceheight: ' + str(sorted(set(source_dataset['sourceheight']))))
print('alum shieldingdensity: ' + str(sorted(set(source_dataset[source_dataset['shielding']=='alum']['shieldingdensity']))))
print('iron shieldingdensity: ' + str(sorted(set(source_dataset[source_dataset['shielding']=='iron']['shieldingdensity']))))
print('lead shieldingdensity: ' + str(sorted(set(source_dataset[source_dataset['shielding']=='lead']['shieldingdensity']))))
print('fwhm: ' + str(sorted(set(source_dataset['fwhm']))))

sourcedist: [50.0, 112.5, 175.0, 237.5, 300.0]
sourceheight: [50.0, 75.0, 100.0, 125.0, 150.0]
alum shieldingdensity: [1.82, 4.18, 7.49, 13.16]
iron shieldingdensity: [1.53, 3.5, 6.28, 11.02]
lead shieldingdensity: [0.22, 0.51, 0.92, 1.61]
fwhm: [6.0, 6.5, 7.0, 7.5, 8.0, 8.5, 9.0]


#### Customize dataset

In [8]:
source_dataset = source_dataset[source_dataset['fwhm']==7.5]
source_dataset = source_dataset[source_dataset['sourcedist']==175.0]
source_dataset = source_dataset[source_dataset['sourceheight']==100.0]

# remove 80% shielding
source_dataset = source_dataset[source_dataset['shieldingdensity']!=13.16]
source_dataset = source_dataset[source_dataset['shieldingdensity']!=11.02]
source_dataset = source_dataset[source_dataset['shieldingdensity']!=1.61]

# remove 60% shielding
source_dataset = source_dataset[source_dataset['shieldingdensity']!=7.49]
source_dataset = source_dataset[source_dataset['shieldingdensity']!=6.28]
source_dataset = source_dataset[source_dataset['shieldingdensity']!=0.92]

# remove 40% shielding
source_dataset = source_dataset[source_dataset['shieldingdensity']!=4.18]
source_dataset = source_dataset[source_dataset['shieldingdensity']!=3.5]
source_dataset = source_dataset[source_dataset['shieldingdensity']!=0.51]

# remove 20% shielding
source_dataset = source_dataset[source_dataset['shieldingdensity']!=1.82]
source_dataset = source_dataset[source_dataset['shieldingdensity']!=1.53]
source_dataset = source_dataset[source_dataset['shieldingdensity']!=0.22]

#### Remove empty spectra

In [9]:
zero_count_indicies = np.argwhere(np.sum(source_dataset.values[:,6:],axis=1) == 0).flatten()

print('indicies dropped: ' +str(zero_count_indicies))

source_dataset.drop(source_dataset.index[zero_count_indicies], inplace=True)

indicies dropped: []


#### Add empty spectra for background 

In [10]:
blank_spectra = []
for fwhm in set(source_dataset['fwhm']):
    num_examples = source_dataset[(source_dataset['fwhm']==fwhm) &
                                  (source_dataset['isotope']==source_dataset['isotope'].iloc()[0])].shape[0]
    for k in range(num_examples):
        blank_spectra_tmp = [0]*1200
        blank_spectra_tmp[5] = fwhm
        blank_spectra_tmp[0] = 'background'
        blank_spectra_tmp[3] = 'background'
        blank_spectra.append(blank_spectra_tmp)

source_dataset = source_dataset.append(pd.DataFrame(blank_spectra,
                                                    columns=source_dataset.columns))

#### Create dataset from spectra

In [11]:
spectra_dataset = source_dataset.values[:,5:].astype('float64')
all_keys = source_dataset['isotope'].values

## Define Training Parameters

#### Define online data augmentation

In [12]:
def integration_time():
    return np.random.uniform(60,600)

def background_cps():
    return np.random.poisson(200)

def signal_to_background():
    return np.random.uniform(0.5,2)

def calibration():
    return [np.random.uniform(0,10),
            np.random.uniform(2500/3000,3500/3000),
            0]

online_data_augmentation = online_data_augmentation_vanilla(background_dataset,
                                background_cps,
                                integration_time,
                                signal_to_background,
                                calibration,)

## Load testing dataset 

In [13]:
testing_dataset = np.load('../dataset_generation/testing_dataset_200keV_1000.npy')

In [14]:
testing_spectra = np.random.poisson(np.add(testing_dataset.item()['sources'], testing_dataset.item()['backgrounds']))
testing_keys = testing_dataset.item()['keys']

mlb=LabelBinarizer()

all_keys_binarized = mlb.fit_transform(all_keys.reshape([all_keys.shape[0],1]))
testing_keys_binarized = mlb.transform(testing_keys)
training_keys_binarized = mlb.transform(all_keys)

## Train network

### Define hyperparameters

In [15]:
number_hyperparameters_to_search = 256
earlystop_errors_test = []
model_id='CNN-onlinedataaugeasy'

In [17]:
def make_model():
    '''
    Makes a random model given some parameters.

    '''
    cnn_filters_choices = ((4, 8),
                       (8, 16),
                       (16, 32),
                       (4,),
                       (8,),
                       (16,),
                       (32,),
                       (4, 8, 16),
                       (8, 16, 32),
                      )

    cnn_kernel_choices = ((2,), (4,), (8,), (16,))
    pool_size_choices = ((2,), (4,), (8,), (16,))

    
    model_features = generate_random_cnn1d_architecture(
        cnn_filters_choices= cnn_filters_choices,
        cnn_kernel_choices=cnn_kernel_choices,
        pool_size_choices=pool_size_choices,
    )
    model_features.trainable = True
    model_features.learining_rate = 10**np.random.uniform(-4,-1)
    model_features.batch_size = 2**np.random.randint(4,6)
    model_features.output_size = training_keys_binarized.shape[1]
    model_features.scaler = choice([make_pipeline(FunctionTransformer(np.log1p, validate=True)),
                                    make_pipeline(FunctionTransformer(np.sqrt, validate=True))])
    model_features.activation_function = choice([tf.nn.relu,tf.nn.tanh,tf.nn.sigmoid])
    model_features.output_function = None
    model_features.Pooling = tf.layers.MaxPooling1D
    model_features.l2_regularization_scale = 10**np.random.uniform(-3,0)
    model_features.dropout_probability = np.random.uniform(0,1)
    model_features.pool_strides = ((2,2,2))
    number_layers = choice([1, 2, 3])
    dense_nodes = 2**np.random.randint(4,8,number_layers)
    dense_nodes = np.sort(dense_nodes)
    dense_nodes = np.flipud(dense_nodes)
    
    model_features.dense_nodes = dense_nodes

    model = CNN1D(model_features)

    return model, model_features 

### Search hyperparameters

In [ ]:
testing_errors = []

for network_id in range(number_hyperparameters_to_search):
    # reset model on each iteration
    model, model_features = make_model()
    optimizer = tf.train.AdamOptimizer(model_features.learining_rate)
    
    costfunction_errors_tmp, earlystop_errors_tmp = train_earlystop(
            training_data=spectra_dataset,
            training_keys=training_keys_binarized,
            testing_data=testing_spectra,
            testing_keys=testing_keys_binarized,
            model=model,
            optimizer=optimizer,
            num_epochs=100,
            obj_cost=model.cross_entropy,
            earlystop_cost_fn=model.f1_error,
            earlystop_patience=5,
            verbose=True,
            fit_batch_verbose=1,
            data_augmentation=online_data_augmentation)
    testing_errors.append(earlystop_errors_tmp)

    # np.save('./final-models/final_test_errors_'+model_id, training_errors)
    # model.save_weights('./final-models/'+model_id+'_checkpoint_'+str(network_id))
    network_id += 1 

Epoch 1: CostFunc loss: 0.00 6.10, EarlyStop loss: 0.00 0.97
Epoch 2: CostFunc loss: 0.00 5.78, EarlyStop loss: 0.00 0.97
Epoch 3: CostFunc loss: 0.00 5.68, EarlyStop loss: 0.00 0.97
Epoch 4: CostFunc loss: 0.00 5.43, EarlyStop loss: 0.00 0.97
Epoch 5: CostFunc loss: 0.00 5.38, EarlyStop loss: 0.00 0.97
Epoch 6: CostFunc loss: 0.00 5.34, EarlyStop loss: 0.00 0.97
Epoch 7: CostFunc loss: 0.00 5.22, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 5.68 Early stopfctn: 5.68
